In [1]:
import os
from dotenv import load_dotenv
import pandas as pd

# Load environment variables from your .env file
# Adjust the path if your .env file is elsewhere
load_dotenv(r"C:\Users\Admin\OneDrive\Documents\GitHub\thesis\backend\.env")

# Get the Supabase credentials from environment variables
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

# Initialize the Supabase client
from supabase import create_client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

In [ ]:
medicines = supabase.table('medicine_medicine').select().execute()
prescriptions = supabase.table('patient_prescription').select().execute()
patient = supabase.table('patient_patient').select().execute()
patient_df = pd.DataFrame(patient.data)
print(patient_df.head(3))
med_df = pd.DataFrame(medicines.data)
pres_df = pd.DataFrame(prescriptions.data)
#print(pres_df['medication_id'].unique())

pres_df['start_date'] = pd.to_datetime(pres_df['sta  nrt_date']) 
pres_df['month'] = pres_df['start_date'].dt.to_period('M').dt.to_timestamp()
pres_df = pres_df.merge(med_df, left_on='medication_id', right_on='id', how='left')

monthly = (
    pres_df
    .groupby(['medication_id', 'name', 'month'])
    .agg({'quantity': 'sum'})
    .reset_index()
)       

          patient_id first_name middle_name last_name           email  \
0  bitancur-02000101         Cj              Bitancur  test@gmail.com   
1           6ND1F8G3      Test1           A   Patient  test@gmail.com   
2           AHLZZZAT      Test2         Ear   Patient  test@gmail.com   

  phone_number date_of_birth street_address    barangay municipal_city  gender  
0  09955471538    2003-07-29       street 1   barangay1    municipal 1    Male  
1  09911456236    2003-05-20       street 1  barangay 1    quezon city  Female  
2    099651681    2014-02-21       street 1   barangay1    municipal 1    Male  


In [9]:
print(pres_df)

    id_x  medication_id  dosage       frequency start_date    end_date  \
0    143           1052  802 mg   Every 4 hours 2025-03-22  2025-03-28   
1    144           1050  333 mg   Every 8 hours 2025-03-21  2025-03-24   
2    145           1056  954 mg     Twice a day 2025-04-19  2025-04-30   
3    146           1050  333 mg     Twice a day 2025-04-19  2025-04-30   
4    150           1056  954 mg     Twice a day 2025-04-29  2025-04-29   
5    151           1056  954 mg     Twice daily 2025-04-29  2025-04-29   
6    152           1048  938 mg     Twice daily 2025-04-29  2025-04-29   
7    153           1056  954 mg     Twice Daily 2025-04-30  2025-04-30   
8    154           1064  861 mg     Twice Daily 2025-04-30  2025-04-30   
9    155           1056  954 mg     Twice daily 2025-05-04  2025-06-04   
10   156           1056  954 mg           twice 2025-05-16  2025-06-16   
11   157           1061  440 mg           Twice 2025-05-29  2025-06-29   
12   158           1056  954 mg       

In [10]:
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
n_splits = 1 
ts = TimeSeriesSplit(n_splits=n_splits)


for medicine_id, group in monthly.groupby('medication_id'):
    group = group.sort_values('month').copy()
    group = group.set_index('month')

    full_month_index = pd.date_range(start=group.index.min(), end=group.index.max(), freq='MS')
    group = group.reindex(full_month_index, fill_value=0)
    group = group.reset_index().rename(columns={'index': 'month'})
    
    group['month_index'] = (group['month'] - group['month'].min()).dt.days // 30
    group['lag_1'] = group['quantity'].shift(1).fillna(0)
    group['lag_2'] = group['quantity'].shift(2).fillna(0)

    features = ['month_index','lag_1', 'lag_2']
    X = group[features].values
    y = group['quantity'].values

    # Skip medicines with too few samples
    if len(X) < n_splits + 1:
        print(f"Skipping medicine {medicine_id} (only {len(X)} samples)")
        continue

    for fold, (train_index, test_index) in enumerate(ts.split(X)):
        X_train, y_train = X[train_index], y[train_index]
        X_test, y_test = X[test_index], y[test_index]
        print(fold,train_index, test_index)

        model = LGBMRegressor(
            random_state=42,
            n_estimators=50,
            learning_rate=0.01,
            max_depth=2,
            num_leaves=31,
            min_child_samples=1
        )
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        mse = mean_squared_error(y_test, preds)
        print(f"[{medicine_id}] Fold {fold+1} MSE: {mse:.2f}")


ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

# Simulate some monthly time series data (e.g., demand)
np.random.seed(42)
n_months = 10

data = pd.DataFrame({
    "month_index": np.arange(n_months),
    "demand": np.random.poisson(lam=20, size=n_months)
})
data["lag_1"] = data["demand"].shift(1).fillna(0)
data["lag_2"] = data["demand"].shift(2).fillna(0)

# Features and target
X = data[["month_index", "lag_1", "lag_2"]].values
y = data["demand"].values

# TimeSeriesSplit config
tscv = TimeSeriesSplit(n_splits=3)

# Tracking best model
best_model = None
best_mse = float("inf")

for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]

    model = LGBMRegressor(random_state=42)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)

    print(f"Fold {fold+1} MSE: {mse:.2f}")

    # Check if this is the best model so far
    if mse < best_mse:
        best_mse = mse
        best_model = model

print(f"\n✅ Best model has MSE: {best_mse:.2f}")

# Use `best_model` for predictions or retrain on full data if needed


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 4, number of used features: 0
[LightGBM] [Info] Start training from score 21.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the spli

c:\Users\Admin\OneDrive\Documents\GitHub\thesis\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Admin\OneDrive\Documents\GitHub\thesis\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\Admin\OneDrive\Documents\GitHub\thesis\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
HTTP/1.1 200 OK
Date: Sat, 23 Aug 2025 02:11:38 GMT
Server: WSGIServer/0.2 CPython/3.13.5
Content-Type: application/json
Vary: Accept, Cookie, origin
Allow: GET, HEAD, OPTIONS
X-Frame-Options: DENY
Content-Length: 667
X-Content-Type-Options: nosniff
Referrer-Policy: same-origin
Cross-Origin-Opener-Policy: same-origin

{
  "results": [
    {
      "medicine_id": 1048,
      "name": "Acetocillin",
      "mse": 23134.0625,
      "r2": -0.1973532684643653,
      "accuracy": -0.45774415535444946,
      "forecast_next_3_months": [
        111.25,
        111.25,
        111.25
      ]
    },
    {
      "medicine_id": 1050,
      "name": "Dextrophen",
      "mse": 132.8125,
      "r2": -0.0042533081285445196,
      "accuracy": 0.9081428479959022,
      "forecast_next_3_months": [
        124.75,
        124.75,
        124.75
      ]
    },
    {
      "medicine_id": 1052,
      "name": "Amoxicillin",
      "mse": 3310.765625,
      "r2": -0.0013657844990548895,
      "accuracy": 0.27751446964327897,
      "forecast_next_3_months": [
        106.375,
        106.375,
        106.375
      ]
    },
    {
      "medicine_id": 1056,
      "name": "Cefcillin",
      "mse": 15438.25,
      "r2": -0.045792477433995504,
      "accuracy": -1.4078557312252964,
      "forecast_next_3_months": [
        180.5,
        180.5,
        180.5
      ]
    }
  ]
}